<a href="https://colab.research.google.com/github/nan-park/cp1_project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0과 1을 분류하는 인공신경망 프로그래밍**

# **Import**

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

# **Data Load**

In [27]:
from google.colab import drive  # google drive mount
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# 데이터 불러오기
def data_load():
  df = pd.read_csv('/content/drive/MyDrive/binary_dataset.csv')
  return df
df = data_load()

In [29]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y
0,136.093750,51.691005,-0.045909,-0.271816,9.342809,38.096400,4.345438,18.673649,0
1,99.367188,41.572202,1.547197,4.154106,27.555184,61.719016,2.208808,3.662680,1
2,100.890625,51.890394,0.627487,-0.026498,3.883779,23.045267,6.953168,52.279440,0
3,120.554688,45.549905,0.282924,0.419909,1.358696,13.079034,13.312141,212.597029,1
4,121.882812,53.042675,0.200521,-0.282219,2.116221,16.580876,8.947603,91.011762,0


# **Data Split**

In [30]:
def minMaxScaler(X: pd.DataFrame, x_max, x_min):
  X = X.copy()
  for feature in x_max.keys():
    X[feature] = (X[feature] - x_min[feature]) / (x_max[feature] - x_min[feature])
  return X

def normalization(X_train, X_test):
  features = list(X_train.columns)
  x_max = {}
  x_min = {}
  for feature in features:
    x_max[feature] = X_train[feature].max()
    x_min[feature] = X_train[feature].min()
  X_train = minMaxScaler(X_train, x_max, x_min)
  X_test = minMaxScaler(X_test, x_max, x_min)
  return X_train, X_test

In [31]:
# 학습/테스트 데이터 뒤섞기
def df_shuffle(df):
  return df.sample(frac=1).reset_index(drop=True)

# 특성, 타겟 나누기
def divide_xy(df):
  target = 'y'
  features = list(df.columns)
  features.remove(target)

  X = df[features]
  y = np.array(df[target]).reshape(-1, 1)
  return X, y

# 학습/테스트 데이터 분리하기
def train_test_divide(X, y, test_size=0.2):  # X: pandas dataframe, y: numpy array
  length = len(y)
  test_index = int(length * test_size)

  X_test = X[:test_index]
  y_test = y[:test_index]

  X_train = X[test_index:]  
  y_train = y[test_index:]
  
  # train 데이터에 맞춰서 normalization(minMaxScaler 이용)
  X_train, X_test = normalization(X_train, X_test)

  return X_train, y_train, X_test, y_test

In [32]:
# 위의 함수 통합
def train_test_split(df, shuffle=True, test_size=0.2):
  if shuffle:
    df = df_shuffle(df)
  
  X, y = divide_xy(df)
  return train_test_divide(X, y, test_size=test_size)

# X_train, y_train, X_test, y_test = train_test_split(df)

In [33]:
# 미니배치 설정
def split_mini_batch(X, y, n): # train, test 들어올 예정
  # 4개씩 미니배치 설정. 나머지는 버리기
  length = len(y)
  num = length // n # 미니배치 개수
  X_batch_list = []
  y_batch_list = []
  for i in range(num):
    i = i * n
    # 비복원 추출. 데이터가 적기 때문에 겹치지 않는 게 나을 듯.
    X_batch_list.append(X[i:i+n]) # index: 0~4, 4~8, 8~12, ...
    y_batch_list.append(y[i:i+n])
  return X_batch_list, y_batch_list

def train_test_mini_batch(X_train, y_train, X_test, y_test, n):
  train_X_batch_list, train_y_batch_list = split_mini_batch(X_train, y_train, n) # train 데이터
  test_X_batch_list, test_y_batch_list = split_mini_batch(X_test, y_test, n) # test 데이터
  return train_X_batch_list, train_y_batch_list, test_X_batch_list, test_y_batch_list

# **Weight and Bias Initialization**

In [34]:
# Xavier 초기화(활성화함수가 시그모이드일 때 잘 동작), 편향 포함한 후에 분리하기
# 이전 층 노드 개수가 n, 현재 층 노드 개수 m일 때, 표준편차가 2/루트(n+m)인 정규분포로 초기화
def initialize_parameter(n: int, m: int):
  init = np.random.normal(0, 2/((n+m)**2), (n+1, m))
  W = init[:-1, :]
  b = init[-1, :]
  return W, b

# initialize_parameter(4, 5)

In [35]:
def sigmoid(x):
    return 1 / (1 +np.exp(-x))

def classification(x):
  if x < 0.5:
    return 0
  else:
    return 1

# **Sequential Layers**

In [36]:
class Layer():  # 입력층, 은닉층, 출력층
  def __init__(self, node_num: int, activation='linear'):
    self.node_num = node_num  # 레이어의 노드 개수
    self.activation = activation  # 활성화 함수
    self.prev = None  # 이전 층
    self.next = None  # 다음 층

  def set_weights(self):  # 가중치 행렬, 편향 초기화
    if self.prev is not None:
      prev_node_num = self.prev.node_num
      self.W, self.b = initialize_parameter(prev_node_num, self.node_num)
  
  # input 행렬 X
  @property
  def X(self):
    return self._X
  @X.setter
  def X(self, value):
    self._X = value

class Dense(Layer): # 은닉층, 출력층
  def __init__(self, node_num, activation='linear'):
    # super().__init__(self, node_num)
    self.node_num = node_num
    self.activation = activation
    self.prev = None
    self.next = None
  
  def output(self):
    answer = np.dot(self._X, self.W) + self.b
    if self.activation == 'linear': # 활성화함수 없음(선형함수)
      return answer
    elif self.activation == 'sigmoid': # 활성화함수 : 시그모이드
      answer = sigmoid(answer)
      return answer

class Input(Layer): # 입력층
  def __init__(self, node_num, activation='linear'):
    self.node_num = node_num
    self.activation = activation
    self.prev = None
    self.next = None
  # 입력층의 경우 input을 그대로 출력한다
  def output(self): 
    return self._X

In [37]:
# Sequential([])에 layer 쌓고 서로 연결되도록 하기. 가중치 초기화 가능해야 함
class Sequential():
  def __init__(self, layer_list: list): # Layer들을 서로 링크드리스트로 연결. 처음과 끝 지정.
    # layer가 없는 경우
    if len(layer_list)==0:
      self.head = None
      self.tail = None
    # layer가 1개인 경우
    elif len(layer_list)==1:
      self.head = layer_list[0]
      self.tail = layer_list[0]
    else: # layer가 2개 이상인 경우
      self.head = layer_list[0]
      iterator = self.head
      for layer in layer_list[1:-1]:
        layer.prev= iterator
        iterator.next = layer
        iterator = layer
      iterator.next = layer_list[-1]
      self.tail = layer_list[-1]
      self.tail.prev = iterator

    # 가중치, 편향 초기화
    iterator = self.head
    while iterator:
      iterator.set_weights()
      iterator = iterator.next

  # input 행렬 X
  @property
  def input(self):
    return self._input
  @input.setter
  def input(self, value):
    self._input = value
    self.head.X = value # 처음 층 input에도 들어가도록 하기

  def output(self):
    iterator = self.head  # Input
    while iterator.next:
      iterator.next.X = iterator.output()
      iterator = iterator.next
    return iterator.output()
    

In [38]:
# model = Sequential([Input(8), Dense(16), Dense(32), Dense(1, activation='sigmoid')])

# **Backpropagation**

In [39]:
# 역전파 학습
def back_prop(model: Sequential, y_pred_prob, y_batch, learning_rate=1):
    def sigmoid_prime(y): # sigmoid의 도함수
        return y * (1 - y)
    def cross_entropy_prime(y_pred_prob, y_batch):  # binary cross entropy의 도함수
        return y_pred_prob - y_batch

    # 출력층부터 시작해서 반대방향으로
    iterator = model.tail
    while iterator != model.head:
        # dError/dY
        if iterator == model.tail:  # 출력층이면
            error = cross_entropy_prime(y_pred_prob, y_batch) # 손실함수의 도함수
        else: # 출력층을 지난 후
            error = np.dot(delta, iterator.next.W.T)
        # dY/dy
        if iterator.activation == 'sigmoid':  # 활성화함수가 sigmoid면
            delta = error * sigmoid_prime(y_pred_prob)
        elif iterator.activation == 'linear': # 활성화함수가 없으면(linear)
            delta = error
        # dy/dw
        delta_mean = np.mean(delta) # 편향 업데이트를 위한 delta
        iterator.b -= learning_rate * delta_mean  # 편향 업데이트
        iterator.W -= learning_rate * np.dot(iterator.X.T, delta) # 가중치 업데이트
        iterator = iterator.prev  # 이전 층으로 이동

# **Predict and Evaluate**

In [40]:
# 정답 예측
def predict(X_batch, y_batch, model: Sequential):
    model.input = X_batch # Sequential의 입력층에 X 입력
    y_pred_prob = model.output()  # 순전파 과정을 통해 마지막 층까지 이동 후 결과 출력
    return y_pred_prob  # 예측 확률

# 미니배치 단위로 evaluate(정확도, 손실함수 출력)
def evaluate_mini_batch(X_batch, y_batch, model, i):
    def accuracy(y_pred, y_batch):  # 정확도
        return sum(y_pred == y_batch)[0] / y_pred.shape[0]

    def cross_entropy(y_pred_prob, y_batch):    # 손실함수
        delta = 1e-7
        return -np.mean(y_batch * np.log(y_pred_prob + delta) + (1 - y_batch) * np.log(1 - y_pred_prob + delta))   # (체크)
    
    # y_pred_prob(예측 확률)로 0과 1로 분류
    def classification(x):  # x: 0~1 사이의 실수
        return 0 if x < 0.5 else 1
        # return round(x, 0)

    classify = np.vectorize(classification)
    y_pred_prob = predict(X_batch, y_batch, model)  # 정답 예측
    y_pred = classify(y_pred_prob)  # 0과 1로 분류
    accuracy = accuracy(y_pred, y_batch)  # 정확도 측정
    cross_entropy = cross_entropy(y_pred_prob, y_batch) # 손실함수(binary cross entropy) 측정
    print(f"[Mini-Batch {i+1}] Loss = {round(cross_entropy, 3)}, Accuracy = {round(accuracy, 3)}")
    back_prop(model, y_pred_prob, y_batch)
    return accuracy, cross_entropy

In [41]:
# epoch 단위로 evaluate(정확도, 손실함수 출력)
def evaluate_epoch(X_batch_list, y_batch_list, model, epoch):
    accuracy_list = []
    cross_entropy_list = []
    for i in range(len(X_batch_list)):
        # 리스트 중 미니 배치 하나 선택
        X_batch = X_batch_list[i]
        y_batch = y_batch_list[i]
        accuracy, cross_entropy = evaluate_mini_batch(X_batch, y_batch, model, i) # 미니 배치 단위로 evaluate
        accuracy_list.append(accuracy)
        cross_entropy_list.append(cross_entropy)

    total_accuracy = np.mean(accuracy_list) # 정확도 평균
    total_cross_entropy = np.mean(cross_entropy_list) # 손실함수 평균
    print(f"[Epoch {epoch+1}] Loss = {round(total_cross_entropy, 3)}, Accuracy = {round(total_accuracy, 3)}\n")

# **Main**

In [42]:
def main():

    # csv 데이터 불러오기
    df = data_load()

    # train, test 데이터로 나누기
    X_train, y_train, X_test, y_test = train_test_split(df)
    # 미니배치(하나에 4개 데이터)로 나누기
    X_batch_list, y_batch_list, test_X_batch_list, test_y_batch_list = train_test_mini_batch(X_train, y_train, X_test, y_test, 4)

    # 모델 만들기(입력층, 은닉층, 출력층)
    model = Sequential([Input(8), Dense(8), Dense(1, activation='sigmoid')])

    # 모델 실행 및 정확도, 손실함수 측정
    print("<Train data>")
    # for i in range(10):
    #     evaluate_epoch(X_batch_list, y_batch_list, model, i)
    evaluate_epoch(X_batch_list, y_batch_list, model, 0)    # epoch 1
    print("<Test data>")
    evaluate_epoch(test_X_batch_list, test_y_batch_list, model, 0)

main()

<Train data>
[Mini-Batch 1] Loss = 0.687, Accuracy = 0.75
[Mini-Batch 2] Loss = 0.65, Accuracy = 1.0
[Mini-Batch 3] Loss = 0.699, Accuracy = 0.5
[Mini-Batch 4] Loss = 0.648, Accuracy = 0.75
[Epoch 1] Loss = 0.671, Accuracy = 0.75

<Test data>
[Mini-Batch 1] Loss = 0.698, Accuracy = 0.5
[Epoch 1] Loss = 0.698, Accuracy = 0.5

